<a href="https://colab.research.google.com/github/abbeymars/AbigailMarsella-AER850Project2/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import os
import pandas as pd
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import layers


#Step 1: Data Processing
#1.1 Define input image shape
input_shape = (500, 500, 3)  #500x500 pixels, 3 color channels (RGB)
batch_size = 32

#1.2 Establish data directories
from google.colab import drive
drive.mount('/content/drive') #mount google drive
zip_file = '/content/drive/My Drive/AER850Proj2/Dataset.zip'
extract = '/content/Dataset/Data/Data'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract) #extract data
#establish train, validation, and test directories
train = os.path.join(extract, 'train')
validation = os.path.join(extract, 'valid')
test = os.path.join(extract, 'test')
#verify data paths are set correctly
print("Training directory:", train)
print("Validation directory:", validation)
print("Test directory:", test)

#1.3 Perform Data Augementation
# Training data
train_data = ImageDataGenerator(
    rescale=1/255, #scale pixel values from [0, 255] to [0, 1]
    shear_range=0.2, #shear intensity
    zoom_range=0.2, #zoom range
    horizontal_flip=True #randomly flip images horizontally
)
# Validation data (note: apply only rescaling for validation)
validation_data = ImageDataGenerator(
    rescale=1/255 #scale pixel values from [0, 255] to [0, 1]
)

#1.4 Create the Train and Validation Generators
#train generator
train_generator = train_data.flow_from_directory(
    train,
    target_size=(500, 500),      #resize images (500x500 pixels)
    batch_size=batch_size,       #batch size 32
    class_mode='categorical',    #class mode categorical
    shuffle=True                 #shuffle images, ensure randomness
)
#validation generator
validation_generator = validation_data.flow_from_directory(
    validation,
    target_size=(500, 500),      #resize images (500x500 pixels)
    batch_size=batch_size,       #batch size 32
    class_mode='categorical',    #class mode categorical
    shuffle=False                #no shuffle, doesn't need to be random
)
#verify generators are set up correctly
print("Class indices:", train_generator.class_indices)
print("Class indices:", validation_generator.class_indices)


#Step 2: Neural Network Architecture Design
#initialize
model = models.Sequential()

#Convolutional Layer 1
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Convolutional Layer 2
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Convolutional Layer 3
model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Convolutional Layer 4
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Flatten layer
model.add(Flatten())

#Fully Connected Dense Layer 1
model.add(Dense(128, activation='relu'))

#Fully Connected Dense Layer 2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  #Dropout layer to reduce overfitting

#Output Layer
model.add(Dense(3, activation='softmax'))  #3 neurons

#Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Display the model's architecture
model.summary()


#Step 3: Hyperparameter Analysis
#Training the model
history = model.fit(
    train_generator,  #Training data generator
    epochs=10,  #Number of epochs to train the model
    batch_size=batch_size,  #Batch size is 32
    validation_data=validation_generator,  #Validation data generator
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training directory: /content/Dataset/Data/Data/train
Validation directory: /content/Dataset/Data/Data/valid
Test directory: /content/Dataset/Data/Data/test
Found 1942 images belonging to 3 classes.
Found 431 images belonging to 3 classes.
Class indices: {'crack': 0, 'missing-head': 1, 'paint-off': 2}
Class indices: {'crack': 0, 'missing-head': 1, 'paint-off': 2}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 498, 498, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 249, 249, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 247, 247, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 123, 123, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 121, 121, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 60, 60, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 460800)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │      29,491,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,584,707 (112.86 MB)

 Trainable params: 29,584,707 (112.86 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.3748 - loss: 3.4157 - val_accuracy: 0.5731 - val_loss: 0.9631
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.4499 - loss: 0.9937 - val_accuracy: 0.6148 - val_loss: 0.8136
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.4945 - loss: 0.9437 - val_accuracy: 0.5267 - val_loss: 0.8448
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.5751 - loss: 0.8791 - val_accuracy: 0.6473 - val_loss: 0.8050
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.5758 - loss: 0.8851 - val_accuracy: 0.6311 - val_loss: 0.7788
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.5818 - loss: 0.8806 - val_accuracy: 0.6636 - val_loss: 0.7501
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.6082 - loss: 0.8005 - val_accuracy: 0.6613 - val_loss: 0.7479
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.5966 - loss: 0.8176 - val_accuracy: 0.6659 - val_loss: 0.